# Main (Avg Layer - All Combinations)

The following sheet contains the list of experiments conducted combining different loss functions and miners, accordingly to what has been described in [our work](https://github.com/Elbarbons/Machine-Learning-and-Deep-Learning-Project/blob/main/MLDL_ProjectPaper.pdf). For all the experiments was adopted  an **Avg layer** as pooling layer.  

Please notice that this document is not supposed to be ran in one go. After completing the introductory steps, each experiment can be carried out singularly.

**Introductory Steps**

Most of the relevant introductory actions can be fulfilled by running the code in the next tab. Please execute the following steps:


*   **Mounting Google Drive:** the project was developed storing all of the necessary data on a Google Drive. Please make sure to mount it correctly before running any other line of code.
*   **Requirements:** a list of libraries must be installed before starting the experiments (at least all the ones in the *requirement.txt* file). To do so, update the path to the actual position of *requirement.txt* in your system.
*   **Datasets:** a few different datasets are used to train, validate and test the nets. Update the path to the actual position in your system of the ones that will be put to use.
*   **Other hardcoded paths:** all dataset classes need a reference to their root folder. Before using a certain dataset, please update this field in its relative document (*TokyoDataset.ipynb* for Tokyo, *SanFranciscoDataset.ipynb* for San Francisco, *GSVCitiesDataset.ipynb* for GSV).
*   **Additional requirements for New Miners:** new miners take advantage of a numpy array called *positive_classes.npy*. Identify it in your system and update the relative variable in the file *Losses.ipynb*.


In [ ]:
# Definition of some PATHS
DRIVE_PATH = '/content/drive'

# Mounting Google Drive
import os
from google.colab import drive

if not os.path.isdir(DRIVE_PATH):
  drive.mount(DRIVE_PATH)

# Install dependencies from requirements.txt
!pip install -q -r /content/drive/MyDrive/Colab_Notebooks/requirements.txt

# Copy notebooks to root path to import them
!cp -r /content/drive/MyDrive/Colab_Notebooks/. /content/

# Unzip datasets in local folder
!unzip -q "/content/drive/MyDrive/Datasets/gsv_xs.zip" -d "/content/Datasets/"
!unzip -q "/content/drive/MyDrive/Datasets/sf_xs.zip" -d "/content/Datasets/"
!unzip -q "/content/drive/MyDrive/Datasets/tokyo_xs.zip" -d "/content/Datasets/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 52.4 MB/s eta 0:00:00
 

# MultiSimilarityMiner - MultiSimilarityLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)


model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc

    agg_arch='AVG',
    agg_config={},

    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,

    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='MultiSimilarityMiner', # Example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner. 
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# Model params saving using Pytorch Lightning
# We save the best 3 models accoring to Recall@1 on SanFrancisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# We instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start the training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss  | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | AdaptiveAVGPool      | 0     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 32.79 | 48.23 | 55.14 | 59.65 | 62.96 | 65.54 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.67 | 71.63 | 77.38 | 80.51 | 82.53 | 84.15 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 57.38 | 71.78 | 77.87 | 80.91 | 83.22 | 84.91 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 19min 30s, sys: 23min 20s, total: 1h 42min 51s
Wall time: 1h 54min 45s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_MULTISIMILARITYMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_MULTISIMILARITYMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720472421.6e30700ae09e (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5738]_R5[0.7178].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5667]_R5[0.7163].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5650]_R5[0.7169].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_MULTISIMILARITYMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5738]_R5[0.7178].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5738]_R5[0.7178].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 22.10 | 37.00 | 43.80 | 46.70 | 49.40 | 51.00 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 40.95 | 57.14 | 64.76 | 68.57 | 72.38 | 74.29 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.221           │           0.221           │
│    SanFrancisco_xs/R10    │           0.438           │           0.438           │
│    SanFrancisco_xs/R5     │           0.37            │           0.37            │
│        Tokyo_xs/R1        │    0.4095238095238095     │    0.4095238095238095     │
│       Tokyo_xs/R10        │    0.6476190476190476     │    0.6476190476190476     │
│        Tokyo_xs/R5        │    0.5714285714285714     │    0.5714285714285714     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.3 s, sys: 22.3 s, total: 1min 7s
Wall time: 58.4 s


# NewMultiSimilarityMiner - MultiSimilarity Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss     | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | AdaptiveAVGPool         | 0     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 32.78 | 48.25 | 55.25 | 59.79 | 63.08 | 65.54 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.55 | 71.75 | 77.31 | 80.75 | 83.04 | 84.54 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 57.31 | 72.13 | 77.99 | 80.95 | 83.15 | 84.92 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 20min 42s, sys: 23min 40s, total: 1h 44min 22s
Wall time: 1h 54min 42s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWMULTISIMILARITYMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_NEWMULTISIMILARITYMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720439183.6e30700ae09e (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5655]_R5[0.7175].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5679]_R5[0.7160].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWMULTISIMILARITYMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 19.50 | 35.50 | 40.90 | 45.20 | 48.50 | 50.80 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 40.00 | 55.87 | 66.35 | 70.48 | 73.33 | 74.60 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.195           │           0.195           │
│    SanFrancisco_xs/R10    │           0.409           │           0.409           │
│    SanFrancisco_xs/R5     │           0.355           │           0.355           │
│        Tokyo_xs/R1        │            0.4            │            0.4            │
│       Tokyo_xs/R10        │    0.6634920634920635     │    0.6634920634920635     │
│        Tokyo_xs/R5        │    0.5587301587301587     │    0.5587301587301587     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.5 s, sys: 22 s, total: 1min 7s
Wall time: 57.3 s


# TripletMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                | Params
---------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss | 0     
1 | miner      | TripletMarginMiner  | 0     
2 | backbone   | ResNet              | 2.8 M 
3 | aggregator | AdaptiveAVGPool     | 0     
---------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 27.01 | 41.07 | 48.87 | 53.23 | 56.89 | 59.58 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 54.00 | 70.45 | 76.02 | 79.58 | 81.93 | 84.12 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 55.30 | 71.04 | 77.19 | 80.43 | 82.87 | 84.55 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 21min 40s, sys: 22min 52s, total: 1h 44min 32s
Wall time: 1h 50min 51s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_TRIPLETMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_TRIPLETMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720446067.6e30700ae09e (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720439183.6e30700ae09e (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5655]_R5[0.7175].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5679]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_TRIPLETMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5530]_R5[0.7104].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5530]_R5[0.7104].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.10 | 32.70 | 38.80 | 42.50 | 45.90 | 48.60 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 33.33 | 55.24 | 63.17 | 66.98 | 69.84 | 72.70 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.201           │           0.201           │
│    SanFrancisco_xs/R10    │           0.388           │           0.388           │
│    SanFrancisco_xs/R5     │           0.327           │           0.327           │
│        Tokyo_xs/R1        │    0.3333333333333333     │    0.3333333333333333     │
│       Tokyo_xs/R10        │    0.6317460317460317     │    0.6317460317460317     │
│        Tokyo_xs/R5        │    0.5523809523809524     │    0.5523809523809524     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.2 s, sys: 22.3 s, total: 1min 7s
Wall time: 57.2 s


# NewTripletMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss   | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | AdaptiveAVGPool       | 0     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 27.21 | 41.42 | 49.06 | 53.42 | 57.01 | 59.69 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 53.45 | 69.31 | 75.22 | 78.88 | 81.03 | 82.91 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 54.11 | 70.15 | 75.87 | 78.82 | 81.22 | 82.96 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 22min 27s, sys: 23min 38s, total: 1h 46min 6s
Wall time: 1h 55min 36s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWTRIPLETMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_NEWTRIPLETMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720446067.6e30700ae09e (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720439183.6e30700ae09e (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5655]_R5[0.7175].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5679]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWTRIPLETMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5411]_R5[0.7015].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5411]_R5[0.7015].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.30 | 31.10 | 37.10 | 41.60 | 44.60 | 47.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 34.29 | 50.79 | 58.10 | 64.44 | 66.98 | 69.52 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.203           │           0.203           │
│    SanFrancisco_xs/R10    │           0.371           │           0.371           │
│    SanFrancisco_xs/R5     │           0.311           │           0.311           │
│        Tokyo_xs/R1        │    0.34285714285714286    │    0.34285714285714286    │
│       Tokyo_xs/R10        │     0.580952380952381     │     0.580952380952381     │
│        Tokyo_xs/R5        │    0.5079365079365079     │    0.5079365079365079     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.4 s, sys: 22.8 s, total: 1min 8s
Wall time: 56 s


# PairMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='PairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on SanFrancisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                | Params
---------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss | 0     
1 | miner      | PairMarginMiner     | 0     
2 | backbone   | ResNet              | 2.8 M 
3 | aggregator | AdaptiveAVGPool     | 0     
---------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 14.24 | 23.07 | 27.97 | 31.58 | 34.07 | 36.41 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.46 | 71.66 | 77.57 | 80.95 | 83.25 | 84.91 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.35 | 71.29 | 77.00 | 80.38 | 82.47 | 83.84 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 20min 29s, sys: 23min 21s, total: 1h 43min 50s
Wall time: 1h 53min 42s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_PAIRMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_PAIRMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720446067.6e30700ae09e (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720439183.6e30700ae09e (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 44%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5655]_R5[0.7175].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5731]_R5[0.7213].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5679]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_PAIRMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5694]_R5[0.7194].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5694]_R5[0.7194].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.00 | 33.00 | 39.00 | 42.80 | 46.00 | 49.10 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 36.83 | 55.87 | 63.17 | 68.57 | 72.70 | 75.24 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │            0.2            │            0.2            │
│    SanFrancisco_xs/R10    │           0.39            │           0.39            │
│    SanFrancisco_xs/R5     │           0.33            │           0.33            │
│        Tokyo_xs/R1        │    0.3682539682539683     │    0.3682539682539683     │
│       Tokyo_xs/R10        │    0.6317460317460317     │    0.6317460317460317     │
│        Tokyo_xs/R5        │    0.5587301587301587     │    0.5587301587301587     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.2 s, sys: 21.7 s, total: 1min 6s
Wall time: 53.8 s


# NewPairMarginMiner - MultiSimilarity Loss - 200 batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='MultiSimilarityLoss',
    miner_name='NewPairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

importing Jupyter notebook from GSVCitiesDataloader.ipynb
importing Jupyter notebook from GSVCitiesDataset.ipynb
importing Jupyter notebook from SanFranciscoDataset.ipynb
importing Jupyter notebook from TokyoDataset.ipynb
importing Jupyter notebook from VPRModel.ipynb
importing Jupyter notebook from Losses.ipynb


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1


importing Jupyter notebook from Validation.ipynb
importing Jupyter notebook from Helper.ipynb
importing Jupyter notebook from Resnet.ipynb
importing Jupyter notebook from gem.ipynb
importing Jupyter notebook from avg.ipynb


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)


+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                | Params
---------------------------------------------------
0 | loss_fn    | MultiSimilarityLoss | 0     
1 | miner      | NewPairMarginMiner  | 0     
2 | backbone   | ResNet              | 2.8 M 
3 | aggregator | AdaptiveAVGPool     | 0     
---------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 14.27 | 23.07 | 27.92 | 31.63 | 34.20 | 36.54 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 54.85 | 70.39 | 76.55 | 79.94 | 82.01 | 84.11 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.12 | 71.58 | 77.63 | 81.00 | 83.25 | 85.05 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 27min 54s, sys: 23min 59s, total: 1h 51min 53s
Wall time: 1h 55min 40s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWPAIRMARGINMINER_MSL.zip /content/LOGS

!cp /content/LOGS_AVG_NEWPAIRMARGINMINER_MSL.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/events.out.tfevents.1720512639.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5569]_R5[0.7142].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5485]_R5[0.7039].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5612]_R5[0.7158].ckpt (deflated 7%)


In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWPAIRMARGINMINER_MSL.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5612]_R5[0.7158].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5612]_R5[0.7158].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.40 | 33.90 | 39.70 | 43.10 | 46.00 | 48.20 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 31.11 | 53.65 | 63.81 | 68.25 | 71.75 | 76.51 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.204           │           0.204           │
│    SanFrancisco_xs/R10    │           0.397           │           0.397           │
│    SanFrancisco_xs/R5     │           0.339           │           0.339           │
│        Tokyo_xs/R1        │    0.3111111111111111     │    0.3111111111111111     │
│       Tokyo_xs/R10        │     0.638095238095238     │     0.638095238095238     │
│        Tokyo_xs/R5        │    0.5365079365079365     │    0.5365079365079365     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.2 s, sys: 23.1 s, total: 1min 9s
Wall time: 1min


# MultiSimiliratyMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)


model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | TripletMarginLoss    | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | AdaptiveAVGPool      | 0     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 42.55 | 57.89 | 65.18 | 69.29 | 71.96 | 74.35 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 58.78 | 73.21 | 78.44 | 81.66 | 83.66 | 85.22 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.55 | 73.75 | 79.06 | 81.82 | 84.06 | 85.86 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 4s, sys: 23min 41s, total: 1h 47min 45s
Wall time: 1h 55min 16s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_MULTISIMILARITYMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_MULTISIMILARITYMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720519641.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5878]_R5[0.7321].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5886]_R5[0.7324].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_MULTISIMILARITYMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.70 | 33.50 | 39.30 | 43.50 | 46.70 | 49.90 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 32.70 | 53.33 | 61.59 | 66.98 | 71.11 | 74.60 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.207           │           0.207           │
│    SanFrancisco_xs/R10    │           0.393           │           0.393           │
│    SanFrancisco_xs/R5     │           0.335           │           0.335           │
│        Tokyo_xs/R1        │     0.326984126984127     │     0.326984126984127     │
│       Tokyo_xs/R10        │    0.6158730158730159     │    0.6158730158730159     │
│        Tokyo_xs/R5        │    0.5333333333333333     │    0.5333333333333333     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.7 s, sys: 24.1 s, total: 1min 9s
Wall time: 56.5 s


# NewMultiSimiliratyMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | TripletMarginLoss       | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | AdaptiveAVGPool         | 0     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 42.52 | 57.96 | 65.17 | 69.31 | 71.99 | 74.40 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 58.95 | 73.36 | 78.58 | 81.66 | 83.67 | 85.11 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 59.38 | 73.80 | 79.08 | 81.86 | 83.82 | 85.67 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 27s, sys: 23min 39s, total: 1h 47min 6s
Wall time: 1h 54min 56s


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720519641.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5878]_R5[0.7321].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5886]_R5[0.7324].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.172052655

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWMULTISIMILIRATYMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5938]_R5[0.7380].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5938]_R5[0.7380].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 20.40 | 33.20 | 39.20 | 43.70 | 46.90 | 49.20 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 34.60 | 52.70 | 61.59 | 66.98 | 71.11 | 75.24 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.204           │           0.204           │
│    SanFrancisco_xs/R10    │           0.392           │           0.392           │
│    SanFrancisco_xs/R5     │           0.332           │           0.332           │
│        Tokyo_xs/R1        │     0.346031746031746     │     0.346031746031746     │
│       Tokyo_xs/R10        │    0.6158730158730159     │    0.6158730158730159     │
│        Tokyo_xs/R5        │     0.526984126984127     │     0.526984126984127     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 22.6 s, total: 1min 8s
Wall time: 55 s


# TripletMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.   
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | TripletMarginLoss  | 0     
1 | miner      | TripletMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | AdaptiveAVGPool    | 0     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 38.41 | 54.20 | 61.22 | 65.53 | 68.80 | 71.35 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.96 | 71.69 | 77.37 | 80.73 | 83.26 | 84.84 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 57.25 | 71.84 | 77.59 | 80.92 | 83.26 | 84.97 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 24min 53s, sys: 25min 5s, total: 1h 49min 59s
Wall time: 2h 2min


In [ ]:
# Zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_TRIPLETMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_TRIPLETMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720519641.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5878]_R5[0.7321].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5886]_R5[0.7324].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.172052655

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_TRIPLETMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5725]_R5[0.7184].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5725]_R5[0.7184].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 16.10 | 31.00 | 36.60 | 41.60 | 45.00 | 47.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.03 | 47.62 | 59.05 | 64.13 | 67.94 | 71.75 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.161           │           0.161           │
│    SanFrancisco_xs/R10    │           0.366           │           0.366           │
│    SanFrancisco_xs/R5     │           0.31            │           0.31            │
│        Tokyo_xs/R1        │    0.26031746031746034    │    0.26031746031746034    │
│       Tokyo_xs/R10        │    0.5904761904761905     │    0.5904761904761905     │
│        Tokyo_xs/R5        │    0.47619047619047616    │    0.47619047619047616    │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.1 s, sys: 24.1 s, total: 1min 10s
Wall time: 59.2 s


# NewTripletMarginMiner - TripletMarginLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    #---------------------
    #---- Aggregator -----
    # agg_arch='CosPlace',
    # agg_config={'in_dim': 512,
    #             'out_dim': 512},
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='TripletMarginLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | TripletMarginLoss     | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | AdaptiveAVGPool       | 0     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 38.22 | 54.08 | 61.24 | 65.47 | 68.62 | 71.19 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 55.97 | 71.34 | 76.82 | 80.32 | 82.50 | 83.96 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 56.77 | 71.71 | 77.21 | 80.63 | 83.02 | 84.44 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 25min 8s, sys: 25min 19s, total: 1h 50min 28s
Wall time: 2h 3min 26s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720519641.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.5878]_R5[0.7321].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.5886]_R5[0.7324].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5955]_R5[0.7375].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.172052655

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWTRIPLETMARGINMINER_TRIPLETLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5677]_R5[0.7171].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_4/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.5677]_R5[0.7171].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 15.80 | 30.30 | 36.40 | 40.30 | 43.50 | 46.40 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 25.71 | 47.62 | 58.73 | 65.08 | 70.48 | 73.33 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.158           │           0.158           │
│    SanFrancisco_xs/R10    │           0.364           │           0.364           │
│    SanFrancisco_xs/R5     │           0.303           │           0.303           │
│        Tokyo_xs/R1        │    0.2571428571428571     │    0.2571428571428571     │
│       Tokyo_xs/R10        │    0.5873015873015873     │    0.5873015873015873     │
│        Tokyo_xs/R5        │    0.47619047619047616    │    0.47619047619047616    │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.4 s, sys: 24.6 s, total: 1min 11s
Wall time: 1min


# MultiSimiliratyMiner - FastAPLoss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)


model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params
----------------------------------------------------
0 | loss_fn    | FastAPLoss           | 0     
1 | miner      | MultiSimilarityMiner | 0     
2 | backbone   | ResNet               | 2.8 M 
3 | aggregator | AdaptiveAVGPool      | 0     
----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.68 | 45.20 | 52.62 | 56.97 | 60.32 | 62.99 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.60 | 75.14 | 80.11 | 83.19 | 85.10 | 86.60 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.23 | 75.35 | 80.13 | 83.14 | 85.09 | 86.61 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 18s, sys: 26min 17s, total: 1h 49min 36s
Wall time: 2h 8min 13s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_MULTISIMILARITYMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_MULTISIMILARITYMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720549283.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6160]_R5[0.7514].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6223]_R5[0.7535].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6115]_R5[0.7443].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_MULTISIMILARITYMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6223]_R5[0.7535].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6223]_R5[0.7535].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 25.20 | 40.40 | 47.50 | 51.80 | 53.30 | 54.70 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 37.46 | 54.92 | 63.49 | 67.94 | 71.43 | 74.29 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.252           │           0.252           │
│    SanFrancisco_xs/R10    │           0.475           │           0.475           │
│    SanFrancisco_xs/R5     │           0.404           │           0.404           │
│        Tokyo_xs/R1        │    0.3746031746031746     │    0.3746031746031746     │
│       Tokyo_xs/R10        │    0.6349206349206349     │    0.6349206349206349     │
│        Tokyo_xs/R5        │    0.5492063492063493     │    0.5492063492063493     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 46.2 s, sys: 24.8 s, total: 1min 10s
Wall time: 1min 1s


# NewMultiSimilarityMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewMultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# We start training
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                    | Params
-------------------------------------------------------
0 | loss_fn    | FastAPLoss              | 0     
1 | miner      | NewMultiSimilarityMiner | 0     
2 | backbone   | ResNet                  | 2.8 M 
3 | aggregator | AdaptiveAVGPool         | 0     
-------------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.49 | 45.60 | 52.67 | 57.25 | 60.59 | 63.07 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.42 | 75.27 | 80.25 | 82.99 | 84.95 | 86.36 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.29 | 75.17 | 80.03 | 83.04 | 85.00 | 86.41 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 22min 31s, sys: 25min 37s, total: 1h 48min 8s
Wall time: 2h 3min 47s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720556985.823fb30735ac (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/events.out.tfevents.1720549283.823fb30735ac (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6160]_R5[0.7514].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6223]_R5[0.7535].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_5/checkpoints/resnet18_epoch(06)_step(2191)_R1[0.6115]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWMULTISIMILARITYMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_6/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6229]_R5[0.7517].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_6/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6229]_R5[0.7517].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.30 | 40.50 | 46.90 | 50.50 | 53.30 | 55.60 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 37.46 | 54.60 | 64.44 | 68.89 | 71.75 | 75.56 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.263           │           0.263           │
│    SanFrancisco_xs/R10    │           0.469           │           0.469           │
│    SanFrancisco_xs/R5     │           0.405           │           0.405           │
│        Tokyo_xs/R1        │    0.3746031746031746     │    0.3746031746031746     │
│       Tokyo_xs/R10        │    0.6444444444444445     │    0.6444444444444445     │
│        Tokyo_xs/R5        │     0.546031746031746     │     0.546031746031746     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.9 s, sys: 23.2 s, total: 1min 9s
Wall time: 1min


# TripletMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='TripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

importing Jupyter notebook from GSVCitiesDataloader.ipynb
importing Jupyter notebook from GSVCitiesDataset.ipynb
importing Jupyter notebook from SanFranciscoDataset.ipynb
importing Jupyter notebook from TokyoDataset.ipynb
importing Jupyter notebook from VPRModel.ipynb
importing Jupyter notebook from Losses.ipynb


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1


importing Jupyter notebook from Validation.ipynb
importing Jupyter notebook from Helper.ipynb
importing Jupyter notebook from Resnet.ipynb
importing Jupyter notebook from gem.ipynb
importing Jupyter notebook from avg.ipynb


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)


+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | FastAPLoss         | 0     
1 | miner      | TripletMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | AdaptiveAVGPool    | 0     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.50 | 45.23 | 52.51 | 56.95 | 60.33 | 63.08 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 60.49 | 74.32 | 79.28 | 82.27 | 84.35 | 85.95 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.55 | 74.93 | 79.88 | 82.97 | 84.72 | 86.09 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 23min 46s, sys: 22min 52s, total: 1h 46min 38s
Wall time: 1h 52min 14s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_TRIPLETMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_TRIPLETMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/events.out.tfevents.1720600351.6ab8fa1dc450 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6155]_R5[0.7493].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6049]_R5[0.7432].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6065]_R5[0.7423].ckpt (deflated 7%)


In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_TRIPLETMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6155]_R5[0.7493].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_0/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6155]_R5[0.7493].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 24.90 | 38.30 | 44.60 | 48.70 | 52.00 | 54.30 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 37.46 | 58.73 | 69.52 | 73.97 | 75.87 | 78.10 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.249           │           0.249           │
│    SanFrancisco_xs/R10    │           0.446           │           0.446           │
│    SanFrancisco_xs/R5     │           0.383           │           0.383           │
│        Tokyo_xs/R1        │    0.3746031746031746     │    0.3746031746031746     │
│       Tokyo_xs/R10        │    0.6952380952380952     │    0.6952380952380952     │
│        Tokyo_xs/R5        │    0.5873015873015873     │    0.5873015873015873     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45 s, sys: 20.4 s, total: 1min 5s
Wall time: 57.5 s


# NewTripletMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewTripletMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | loss_fn    | FastAPLoss            | 0     
1 | miner      | NewTripletMarginMiner | 0     
2 | backbone   | ResNet                | 2.8 M 
3 | aggregator | AdaptiveAVGPool       | 0     
-----------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 30.20 | 45.43 | 52.76 | 57.31 | 60.43 | 63.04 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.19 | 74.35 | 79.22 | 82.30 | 84.36 | 85.76 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 61.64 | 74.24 | 79.22 | 82.26 | 84.12 | 85.50 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 20min 30s, sys: 22min 59s, total: 1h 43min 29s
Wall time: 1h 51min 5s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720607144.6ab8fa1dc450 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6083]_R5[0.7378].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6164]_R5[0.7424].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6119]_R5[0.7435].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_0/ (stored 0%)
  adding: conten

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWTRIPLETMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6164]_R5[0.7424].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6164]_R5[0.7424].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 24.00 | 37.90 | 44.70 | 49.10 | 52.00 | 54.30 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 40.00 | 57.78 | 66.67 | 72.38 | 76.51 | 77.14 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.24            │           0.24            │
│    SanFrancisco_xs/R10    │           0.447           │           0.447           │
│    SanFrancisco_xs/R5     │           0.379           │           0.379           │
│        Tokyo_xs/R1        │            0.4            │            0.4            │
│       Tokyo_xs/R10        │    0.6666666666666666     │    0.6666666666666666     │
│        Tokyo_xs/R5        │    0.5777777777777777     │    0.5777777777777777     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 44.9 s, sys: 21.2 s, total: 1min 6s
Wall time: 54.9 s


# PairMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)

model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='PairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params
-----------------------------------------------
0 | loss_fn    | FastAPLoss      | 0     
1 | miner      | PairMarginMiner | 0     
2 | backbone   | ResNet          | 2.8 M 
3 | aggregator | AdaptiveAVGPool | 0     
-----------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 29.75 | 44.40 | 51.66 | 56.09 | 59.13 | 61.68 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.42 | 75.60 | 80.57 | 83.07 | 84.80 | 86.34 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.58 | 75.74 | 80.52 | 83.01 | 84.96 | 86.49 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 19min 40s, sys: 23min 19s, total: 1h 42min 59s
Wall time: 1h 52min 45s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_PAIRMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_PAIRMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720613811.6ab8fa1dc450 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720607144.6ab8fa1dc450 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6083]_R5[0.7378].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6164]_R5[0.7424].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6119]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_PAIRMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6258]_R5[0.7574].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_2/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6258]_R5[0.7574].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 25.30 | 40.80 | 47.30 | 51.70 | 54.70 | 56.40 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 41.59 | 61.59 | 70.16 | 73.97 | 78.73 | 82.54 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.253           │           0.253           │
│    SanFrancisco_xs/R10    │           0.473           │           0.473           │
│    SanFrancisco_xs/R5     │           0.408           │           0.408           │
│        Tokyo_xs/R1        │    0.4158730158730159     │    0.4158730158730159     │
│       Tokyo_xs/R10        │    0.7015873015873015     │    0.7015873015873015     │
│        Tokyo_xs/R5        │    0.6158730158730159     │    0.6158730158730159     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 44.9 s, sys: 23.2 s, total: 1min 8s
Wall time: 56.7 s


# NewPairMarginMiner - FastAP Loss - 200 Batch

In [ ]:
import import_ipynb
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim import lr_scheduler

import GSVCitiesDataloader
import VPRModel

pl.utilities.seed.seed_everything(seed=1, workers=True)

# the datamodule contains train and validation dataloaders,
# refer to ./dataloader/GSVCitiesDataloader.py for details
# if you want to train on specific cities, you can comment/uncomment
# cities from the list TRAIN_CITIES

datamodule = GSVCitiesDataloader.GSVCitiesDataModule(
    batch_size=200,
    img_per_place=4,
    min_img_per_place=4,
    # cities=['London', 'Boston', 'Melbourne'], # you can sppecify cities here or in GSVCitiesDataloader.py
    shuffle_all=False, # shuffle all images or keep shuffling in-city only
    random_sample_from_each_place=True,
    image_size=(320, 320),
    num_workers=8,
    show_data_stats=True,
    val_set_names=['SanFrancisco_val'], # datasets for test for validation, only SanFrancisco-xs implemented
    test_set_names=['SanFrancisco_xs', 'Tokyo_xs'] # datasets for test, SanFrancisco-xs and Tokyo-xs available for testing
)


model = VPRModel.VPRModel(

    #-------------------------------
    #---- Backbone architecture ----
    backbone_arch='resnet18',
    pretrained=True,
    layers_to_freeze=0,
    layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
    agg_arch='AVG',
    agg_config={},
    #-----------------------------------
    #---- Training hyperparameters -----
    #
    lr=0.0002, # 0.03 for sgd
    optimizer='adam', # sgd, adam or adamw
    weight_decay=0, # 0.001 for sgd or 0.0 for adam
    momentum=0.9,
    warmpup_steps=600,
    milestones=[5, 10, 15, 25],
    lr_mult=0.3,
    #---------------------------------
    #---- Training loss function -----
    # see utils.losses.py for more losses
    # example: ContrastiveLoss, TripletMarginLoss, MultiSimilarityLoss,
    # FastAPLoss, CircleLoss, SupConLoss,
    #
    loss_name='FastAPLoss',
    miner_name='NewPairMarginMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
    # New miners leveraging our novelty: NewTripletMarginMiner, NewMultiSimilarityMiner, NewPairMarginMiner.
    miner_margin=0.1,
    faiss_gpu=False
)

# model params saving using Pytorch Lightning
# we save the best 3 models accoring to Recall@1 on San Francisco val
checkpoint_cb = ModelCheckpoint(
    monitor='SanFrancisco_val/R1',
    filename=f'{model.encoder_arch}' +
    '_epoch({epoch:02d})_step({step:04d})_R1[{SanFrancisco_val/R1:.4f}]_R5[{SanFrancisco_val/R5:.4f}]',
    auto_insert_metric_name=False,
    save_weights_only=True,
    save_top_k=3,
    mode='max',)


#------------------
# we instanciate a trainer
trainer = pl.Trainer(
    accelerator='gpu', devices=[0],
    default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz
    num_sanity_val_steps=0, # runs N validation steps before stating training
    precision=16, # we use half precision to reduce  memory usage (and 2x speed on RTX)
    max_epochs=10,
    check_val_every_n_epoch=1, # run validation every epoch
    callbacks=[checkpoint_cb],# we run the checkpointing callback (you can add more)
    reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
    log_every_n_steps=20,
    #fast_dev_run=True # comment if you want to start training the network and saving checkpoints
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time

# we call the trainer, and give it the model and the datamodule
# now you see the modularity of Pytorch Lighning?
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type               | Params
--------------------------------------------------
0 | loss_fn    | FastAPLoss         | 0     
1 | miner      | NewPairMarginMiner | 0     
2 | backbone   | ResNet             | 2.8 M 
3 | aggregator | AdaptiveAVGPool    | 0     
--------------------------------------------------
2.8 M     Trainable params
9.5 K     Non-trainable params
2.8 M     Total params
5.566     Total estimated model params size (MB)



+----------------------+
|   Training Dataset   |
+-------------+--------+
| # of cities | 23     |
| # of places | 62514  |
| # of images | 524701 |
+-------------+--------+

+-------------------------------------+
|         Validation Datasets         |
+------------------+------------------+
| Validation set 1 | SanFrancisco_val |
+------------------+------------------+

+-------------------------------+
|        Training config        |
+------------------+------------+
| Batch size (PxK) | 200x4      |
| # of iterations  | 312        |
| Image size       | (320, 320) |
+------------------+------------+


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 29.91 | 44.60 | 51.85 | 56.32 | 59.49 | 62.09 |
+----------+-------+-------+-------+-------+-------+-------+





...

Validation: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.32 | 75.98 | 80.47 | 83.30 | 84.65 | 86.09 |
+----------+-------+-------+-------+-------+-------+-------+





Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.




+----------------------------------------------------------+
|        Performance on Validation SanFrancisco_val        |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 62.44 | 75.93 | 80.81 | 83.17 | 85.01 | 86.34 |
+----------+-------+-------+-------+-------+-------+-------+



CPU times: user 1h 19min 16s, sys: 22min 54s, total: 1h 42min 10s
Wall time: 1h 50min 51s


In [ ]:
# Cell to zip logs and best models' weights, and save it in Google Drive for future runs
!zip -r /content/LOGS_AVG_NEWPAIRMARGINMINER_FASTAPLOSS.zip /content/LOGS

!cp /content/LOGS_AVG_NEWPAIRMARGINMINER_FASTAPLOSS.zip /content/drive/MyDrive/

  adding: content/LOGS/ (stored 0%)
  adding: content/LOGS/resnet18/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/hparams.yaml (deflated 43%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720613811.6ab8fa1dc450 (deflated 45%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/events.out.tfevents.1720607144.6ab8fa1dc450 (deflated 64%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(07)_step(2504)_R1[0.6083]_R5[0.7378].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6164]_R5[0.7424].ckpt (deflated 7%)
  adding: content/LOGS/resnet18/lightning_logs/version_1/checkpoints/resnet18_epoch(08)_step(2817)_R1[0.6119]_

In [ ]:
%%time


import os
import zipfile
from pytorch_lightning.callbacks import BasePredictionWriter

# ------------------
# we instantiate a trainer
test_trainer = pl.Trainer(
    accelerator='gpu', devices=1,
    default_root_dir=f'./LOGS/{model.encoder_arch}',  # Tensorflow can be used to viz

    precision=16,  # we use half precision to reduce  memory usage (and 2x speed on RTX)
    log_every_n_steps=20,
)

# Trying to load best weights, if it is not possible load weights from previuous training
try:
  trainer.test(model=model, ckpt_path="best", datamodule=datamodule)

except:
  CHECKPOINTS_DIR = "/content/LOGS/Resnet-18/lightning_logs/version_0/checkpoints/"
  if not os.path.isdir('LOGS'):
      with zipfile.ZipFile('/content/drive/MyDrive/LOGS_AVG_NEWPAIRMARGINMINER_FASTAPLOSS.zip', 'r') as zip_ref:
          zip_ref.extractall()

  models = os.listdir(CHECKPOINTS_DIR)

  test_trainer.test(model=model, ckpt_path=CHECKPOINTS_DIR+models[0], datamodule=datamodule)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6244]_R5[0.7593].ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at LOGS/resnet18/lightning_logs/version_3/checkpoints/resnet18_epoch(09)_step(3130)_R1[0.6244]_R5[0.7593].ckpt


+------------------------------+
|        Test Datasets         |
+------------+-----------------+
| Test set 1 | SanFrancisco_xs |
| Test set 2 | Tokyo_xs        |
+------------+-----------------+



Testing: 0it [00:00, ?it/s]



+----------------------------------------------------------+
|           Performance on Test SanFrancisco_xs            |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 26.00 | 41.70 | 48.60 | 52.50 | 54.10 | 55.50 |
+----------+-------+-------+-------+-------+-------+-------+


+----------------------------------------------------------+
|               Performance on Test Tokyo_xs               |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 40.00 | 60.32 | 68.57 | 75.56 | 78.73 | 81.27 |
+----------+-------+-------+-------+-------+-------+-------+





┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    SanFrancisco_xs/R1     │           0.26            │           0.26            │
│    SanFrancisco_xs/R10    │           0.486           │           0.486           │
│    SanFrancisco_xs/R5     │           0.417           │           0.417           │
│        Tokyo_xs/R1        │            0.4            │            0.4            │
│       Tokyo_xs/R10        │    0.6857142857142857     │    0.6857142857142857     │
│        Tokyo_xs/R5        │    0.6031746031746031     │    0.6031746031746031     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

CPU times: user 45.4 s, sys: 23 s, total: 1min 8s
Wall time: 56.9 s


# VISUALIZATIONS

Useful code to visualize the predictive results of the freshly trained network

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#set up the name of the test dataset
name = 'SanFrancisco_xs'
dr_name = 'sf_xs'

#predictions are computed
preds = model.custom_prediction(name)

test_ds = model.trainer.datamodule.test_datasets[model.trainer.datamodule.test_set_names.index(name)]

#choose which image to show (use its index id)
#and take a look at its predictions
#correct ones will be showed with a green title
query_index = 100

img = mpimg.imread('/content/Datasets/'+ dr_name +'/test/queries/'+test_ds.qImages[query_index])
plt.imshow(img)
plt.axis('off')
plt.title('Query')
plt.show()

fig = plt.figure(figsize=(25, 25))
for i in range(0,25):
  img2 = mpimg.imread('/content/Datasets/'+ dr_name +'/test/database/'+test_ds.dbImages[int(preds[query_index,i])])
  rep = fig.add_subplot(5,5,i+1)
  rep.set_title('Prediction #' + str(i+1))
  if np.isin(preds[query_index], test_ds.ground_truth[query_index])[i]:
    rep.title.set_color('green')
    rep.title.set_fontweight('demibold')
  plt.imshow(img2)
  plt.axis('off')

plt.show()

